In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!kaggle kernels output kwahnguyen/baseline-protein-prediction -p ./cafa6-test/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [1]:
from kaggle_secrets import UserSecretsClient
import os 

user_secrets = UserSecretsClient()
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("KAGGLE_KEY")
os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("KAGGLE_USERNAME")

In [2]:
!kaggle competitions download -c cafa-6-protein-function-prediction

  0%|                                               | 0.00/91.3M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 91.3M/91.3M [00:00<00:00, 1.60GB/s]


In [3]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.6 MB/s eta 0:00:00:00:01


In [4]:
import zipfile
import os

zip_path = "cafa-6-protein-function-prediction.zip"
extract_dir = "./cafa6_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

os.listdir(extract_dir)

['IA.tsv', 'Test', 'sample_submission.tsv', 'Train']

In [5]:
import os

dir_path = './cafa6_data/Train'
os.listdir(dir_path)

['train_taxonomy.tsv',
 'go-basic.obo',
 'train_terms.tsv',
 'train_sequences.fasta']

In [6]:
# Calculating Naive Frequency given a sequence
def naive_freq(sequence):
    """
    Input: an amino acid sequence of variable length
    Output: a list of shape (1, 20), each element standing for an amino acid and its frequency
    """
    # Counting the appearance of each amino acid within the sequence
    AMINO_ACIDS = 'ARNDCQEGHILKMFPSTWYV'

    AA_TO_INDEX = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

    frequency_vector = [0] * 20

    for amino_acid in sequence:
        try:
            index = AA_TO_INDEX[amino_acid]
            frequency_vector[index] += 1
        except KeyError:
            print(f"Warning: Skipping unknown amino acid {amino_acid}")
            pass

    # Calculating frequencies for each amino acid
    total_length = len(sequence)

    normalized_vector = frequency_vector
    if total_length > 0:
        normalized_vector = [count / total_length for count in frequency_vector]

    return normalized_vector

In [7]:
CTD_GROUPS_BY_LIST = {
    "Hydrophobicity": {
        1: {"A", "V", "L", "I", "M", "F", "W", "C"},               # Hydrophobic
        2: {"G", "H", "Y", "P", "T", "S"},          # Neutral
        3: {"R", "K", "Q", "E", "D", "N"}           # Polar
    },

    "Charge": {
        1: {"D", "E"},      # Negative
        2: {"A", "G", "I", "L", "M", "F", "P", "Q", "S", "T", "W", "Y", "V", "N", "C"}, # Neutral
        3: {"K", "R", "H"}     # Positive
    },

    "VanDerWaals": {
        1: {"A", "G", "S", "C"},  # Small
        2: {"T", "D", "P", "N", "V"},   # Medium
        3: {"E", "Q", "L", "I", "F", "Y", "M", "H", "K", "R", "W"}   # Large
    },

    "Polarity": {
        1: {"L", "A", "W", "F", "C", "M", "V", "I", "Y"},   # Small
        2: {"P", "T", "S", "G", "H"},    # Medium
        3: {"Q", "N", "E", "D", "K", "R"}   # High
    },

    "Polarizability": {
        1: {"G", "A", "S", "D", "C"},   # Small
        2: {"T", "P", "N", "H", "E", "Q", "K"},   # Medium
        3: {"M", "I", "L", "V", "F", "Y", "W", "R"}
    },

    "SecondStructure": {
        1: {"E", "A", "L", "M", "Q", "K", "R", "H"},  # Helix
        2: {"V", "I", "Y", "C", "W", "F", "T"},  # Strand
        3: {"G", "N", "P", "S", "D"}  # Coil
    },

    "Solvent": {
        1: {"A", "L", "F", "C", "G", "I", "V", "W"},  # Buried
        2: {"R", "K", "Q", "E", "D", "N"},  # Intermediate
        3: {"M", "S", "P", "T", "H", "Y"}  # Exposed
    }
}

def get_group(aa, property_map):
    for g, aa_set in property_map.items():
        if aa in aa_set:
            return g
    return None